In [46]:
import datasets
import pandas as pd

eval_ds = datasets.load_dataset("gaia-benchmark/GAIA", "2023_all")["validation"]
eval_ds = eval_ds.rename_columns(
    {"Question": "question", "Final answer": "true_answer", "Level": "task"}
)


def preprocess_file_paths(row):
    if len(row["file_name"]) > 0:
        row["file_name"] = "data/gaia/validation/" + row["file_name"]

    return row


eval_ds = eval_ds.map(preprocess_file_paths)

eval_df = pd.DataFrame(eval_ds)

Using the latest cached version of the module from /Users/aymeric/.cache/huggingface/modules/datasets_modules/datasets/gaia-benchmark--GAIA/ec492fe4320ee795b1aed6bb46229c5f693226b0f1316347501c24b4baeee005 (last modified on Tue May 28 10:04:32 2024) since it couldn't be found locally at gaia-benchmark/GAIA, or remotely on the Hugging Face Hub.


In [47]:
eval_df["task"].value_counts()

task
2    86
1    53
3    26
Name: count, dtype: int64

In [48]:
eval_df["extension"] = eval_df["file_name"].apply(lambda x: x.split(".")[-1])

In [49]:
eval_df["extension"].value_counts()

extension
          127
xlsx       13
png         8
mp3         3
pdf         3
zip         2
jpg         2
pdb         1
jsonld      1
docx        1
txt         1
pptx        1
csv         1
py          1
Name: count, dtype: int64

### Prepare submission

In [1]:
import datasets
import pandas as pd

pd.set_option("max_colwidth", None)

SET = "test"

eval_ds = datasets.load_dataset("gaia-benchmark/GAIA", "2023_all")[SET]
eval_ds = eval_ds.rename_columns(
    {"Question": "question", "Final answer": "true_answer", "Level": "task"}
)
eval_df = pd.DataFrame(eval_ds)

Using the latest cached version of the module from /Users/aymeric/.cache/huggingface/modules/datasets_modules/datasets/gaia-benchmark--GAIA/ec492fe4320ee795b1aed6bb46229c5f693226b0f1316347501c24b4baeee005 (last modified on Tue May 28 10:04:32 2024) since it couldn't be found locally at gaia-benchmark/GAIA, or remotely on the Hugging Face Hub.


In [2]:
results = pd.read_json(
    f"output_gaia/{SET}/react_code_gpt4o_23-june_planning2_newprompt5_test.jsonl",
    lines=True,
)

In [3]:
from scripts.evaluation.gaia_scorer import question_scorer

results["is_correct"] = results.apply(
    lambda x: question_scorer(x["prediction"], x["true_answer"]), axis=1
)

In [4]:
results["is_correct"].mean()

0.0033333333333333335

In [5]:
results.loc[results["is_correct"], ["question", "prediction"]]

,question,prediction
278,Reverse this ascii picture so that the fish is facing the opposite direction:\n>>$()>. Return the characters (without quotes) in a comma separated list.,".,>,),(,$,>,>"


In [6]:
def get_elements(x):
    output = x[0]["task"]
    for y in x[1:]:
        try:
            if "observation" in y:
                output += y["llm_output"] + "\nObservation:" + y["observation"]
            else:
                output += y["llm_output"] + "\Error:" + str(y["error"])
        except:
            pass
    return output


results["reasoning_trace"] = results["intermediate_steps"].apply(
    lambda x: get_elements(x)
)

In [7]:
results = results[["question", "prediction", "reasoning_trace"]].rename(
    columns={"prediction": "model_answer"}
)

In [8]:
results = results.merge(eval_df[["task_id", "question"]], on="question")
results = results.drop_duplicates(subset=["task_id"])
print(len(results))

300


In [9]:
results = results[
    ["task_id", "model_answer"]
]  # ["task_id", "model_answer", "reasoning_trace"]

In [11]:
results.to_json(
    f"gaia_{SET}_transformers_agents_gpt4o.jsonl", orient="records", lines=True
)

In [56]:
RetrieverTool(vectordb)(query="What is the capital of France?")

'Retrieved documents:\n\nat+the+top+of+the+tower+in+1957%2C+it+is+now+taller+than+the+Chrysler+Building+by+5.2+metres+%2817+ft%29.+Excluding+transmitters%2C+the+Eiffel+Tower+is+the+second+tallest+free-standing+structure+in+France+after+the+Millau+Viaduct)\n===Document===\nBART](https://huggingface.co/facebook/bart-large-cnn?text=The+tower+is+324+metres+%281%2C063+ft%29+tall%2C+about+the+same+height+as+an+81-storey+building%2C+and+the+tallest+structure+in+Paris.+Its+base+is+square%2C+measuring+125+metres+%28410+ft%29+on+each+side.+During+its+construction%2C+the+Eiffel+Tower+surpassed+the+Washington+Monument+to+become+the+tallest+man-made+structure+in+the+world%2C+a+title+it+held+for+41+years+until+the+Chrysler+Building+in+New+York+City+was+finished+in+1930.+It+wa\n===Document===\nВ области NLP ( Обработка текстов на естественном языке ):\n- [Маскированное заполнение слов с помощью BERT](https://huggingface.co/bert-base-uncased?text=Paris+is+the+%5BMASK%5D+of+France)\n- [Распознавание су

In [12]:
from transformers.agents import Tool

retriever_tool = Tool.from_hub(
    repo_id="m-ric/langchain-retriever-tool", vectordb=vectordb, all_sources=all_sources
)

other_tool = Tool.from_hub(repo_id="m-ric/hf-model-downloads")

{'vectordb': <langchain_community.vectorstores.faiss.FAISS object at 0x3acdc95d0>, 'all_sources': ['datasets', 'evaluate', 'blog', 'hf-endpoints-documentation', 'diffusers', 'datasets-server', 'transformers', 'optimum', 'pytorch-image-models', 'gradio', 'deep-rl-class', 'course', 'peft', 'hub-docs']}
{}


In [10]:
retriever_tool(query="What is the capital of France?")

'Retrieved documents:\n\nat+the+top+of+the+tower+in+1957%2C+it+is+now+taller+than+the+Chrysler+Building+by+5.2+metres+%2817+ft%29.+Excluding+transmitters%2C+the+Eiffel+Tower+is+the+second+tallest+free-standing+structure+in+France+after+the+Millau+Viaduct)\n===Document===\nBART](https://huggingface.co/facebook/bart-large-cnn?text=The+tower+is+324+metres+%281%2C063+ft%29+tall%2C+about+the+same+height+as+an+81-storey+building%2C+and+the+tallest+structure+in+Paris.+Its+base+is+square%2C+measuring+125+metres+%28410+ft%29+on+each+side.+During+its+construction%2C+the+Eiffel+Tower+surpassed+the+Washington+Monument+to+become+the+tallest+man-made+structure+in+the+world%2C+a+title+it+held+for+41+years+until+the+Chrysler+Building+in+New+York+City+was+finished+in+1930.+It+wa\n===Document===\nВ области NLP ( Обработка текстов на естественном языке ):\n- [Маскированное заполнение слов с помощью BERT](https://huggingface.co/bert-base-uncased?text=Paris+is+the+%5BMASK%5D+of+France)\n- [Распознавание су

In [13]:
from transformers.agents import HfEngine, ReactJsonAgent

llm_engine = HfEngine("meta-llama/Meta-Llama-3-70B-Instruct")

agent = ReactJsonAgent(
    tools=[RetrieverTool(vectordb, all_sources)], llm_engine=llm_engine
)

agent_output = agent.run("Please show me a LORA finetuning script")

print("Final output:")
print(agent_output)

======== New task ========
Please show me a LORA finetuning script
Calling tool: 'retriever' with arguments: {'query': 'LORA finetuning script', 'source': "['transformers', 'hf-endpoints-documentation']"}
Calling tool: 'retriever' with arguments: {'query': 'LORA finetuning script'}
Calling tool: 'retriever' with arguments: {'query': 'LORA finetuning script', 'source': 'github.com/huggingface/diffusers/blob/main/examples/text_to_image/text_to_image_lora.py'}
Calling tool: 'final_answer' with arguments: {'answer': 'https://github.com/huggingface/diffusers/blob/main/examples/text_to_image/text_to_image_lora.py'}


Final output:
https://github.com/huggingface/diffusers/blob/main/examples/text_to_image/text_to_image_lora.py


In [1]:
!pip install lavague

  Using cached langchain-0.1.20-py3-none-any.whl.metadata (13 kB)
  Using cached gradio_client-0.15.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached langchain_community-0.0.38-py3-none-any.whl.metadata (8.7 kB)
  Using cached langchain_core-0.1.52-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiohttp-3.9.5-cp310-cp310-macosx_11_0_arm64.whl.metadata (7.5 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.4/79.4 kB 7.4 MB/s eta 0:00:00
  Using cached lxml-5.2.2-cp310-cp310-macosx_10_9_universal2.whl.metadata (3.4 kB)
  Using cached charset_normalizer-3.3.2-cp310-cp310-macosx_11_0_arm64.whl.metadata (33 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 26.4 MB/s eta 0:00:0031m27.6 MB/s eta 0:00:01
Using cached gradio_client-0.15.1-py3-none-any.whl (313 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 793.8/793.8 kB 30.3 MB/s eta 0:00:00
Using cached langchain-0.1.20-py3-none-any.whl (1.0 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 27.4 MB/s eta 0:00:00 

In [3]:
from lavague.drivers.selenium import SeleniumDriver
from lavague.core import ActionEngine, WorldModel
from lavague.core.agents import WebAgent

# Set up our three key components: Driver, Action Engine, World Model
driver = SeleniumDriver(headless=False)
action_engine = ActionEngine(driver)
world_model = WorldModel()

# Create Web Agent
agent = WebAgent(world_model, action_engine)

# Set URL
agent.get(
    "https://www.researchgate.net/publication/381296514_Spirituality_Culture_In_Organization_and_Impact_on_Employee_Well-Being"
)

# Run agent with a specific objective
agent.run("Give me the content of page 74 from the paper")

2024-07-02 17:42:25,315 - INFO - Screenshot folder cleared
2024-07-02 17:42:31,103 - INFO - Thoughts:
- The current screenshot shows a page from ResearchGate with a paper titled "Spirituality Culture In Organization and Impact on Employee Well-Being."
- The objective is to provide the content of page 74 from this paper.
- The paper is available for download as a PDF.
- The next step is to download the full-text PDF to access page 74.

Next engine: Navigation Engine
Instruction: Click on the "Download full-text PDF" button.
2024-07-02 17:42:46,978 - ERROR - Navigation error: Message: element click intercepted: Element <a href="https://www.researchgate.net/profile/Ashish-Waghe/publication/381296514_Spirituality_Culture_In_Organization_and_Impact_on_Employee_Well-Being/links/6666bf5785a4ee7261b387e8/Spirituality-Culture-In-Organization-and-Impact-on-Employee-Well-Being.pdf" class="nova-legacy-c-button nova-legacy-c-button--align-center nova-legacy-c-button--radius-m nova-legacy-c-button--

ActionResult(instruction=None, code='from selenium import webdriver\nfrom selenium.webdriver.common.by import By\nfrom selenium.webdriver.chrome.options import Options\nfrom selenium.webdriver.common.keys import Keys\nfrom selenium.webdriver.common.action_chains import ActionChains\n\nelse:\nchrome_options = Options()\nchrome_options.add_argument("--no-sandbox")\nchrome_options.page_load_strategy = (\n)\n\ndriver = webdriver.Chrome(options=chrome_options)\n\ndriver.set_window_size(1080, 1080)\nviewport_height = driver.execute_script("return window.innerHeight;")\nheight_difference = 1080 - viewport_height\ndriver.set_window_size(1080, 1080 + height_difference)\n\ndriver.get("https://www.researchgate.net/publication/381296514_Spirituality_Culture_In_Organization_and_Impact_on_Employee_Well-Being")\n[\n    {\n        "action": {\n            "name": "click",\n            "args": {\n                "xpath": "/html/body/div[1]/div/div/div/div/div[2]/button[3]"\n            }\n        }\n  

In [6]:
example_blob = """Thought: This is an example thought.
Action:
{
    "action": "some_action",
    "action_input": "abc"
}
<end_action>"""

import re


def validate_output(output, grammar):
    return re.match(grammar, output) is not None


from transformers.agents.agents import DEFAULT_JSON_GRAMMAR

is_valid = validate_output(example_blob, DEFAULT_JSON_GRAMMAR)
print(f"Is the output valid? {is_valid}")

Is the output valid? True


In [2]:
from dotenv import load_dotenv
from huggingface_hub import login
from transformers import ReactJsonAgent, HfEngine, Tool
import os

# to load SerpAPI key
load_dotenv()
login(os.getenv("HUGGINGFACEHUB_API_TOKEN"))

llm_engine = HfEngine(model="meta-llama/Meta-Llama-3.1-70B-Instruct")

agent = ReactJsonAgent(
    llm_engine=llm_engine,
    tools=[],
    max_iterations=10,
)
agent.run("How much is 2+16?")

======== New task ========
How much is 2+16?


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /Users/aymeric/.cache/huggingface/token
Login successful


==== Agent is executing the code below:
result = 2 + 16
final_answer(answer=result)
====
Print outputs:

>>> Final answer:
18


18

In [1]:
import os
import gradio as gr
from transformers import ReactJsonAgent, HfEngine, Tool, stream_to_gradio
from gradio import (
    Chatbot,
    ChatMessage,
)
from dotenv import load_dotenv
from huggingface_hub import login

# to load SerpAPI key
load_dotenv()
login(os.getenv("HUGGINGFACEHUB_API_TOKEN"))

llm_engine = HfEngine(model="meta-llama/Meta-Llama-3-70B-Instruct")

agent = ReactJsonAgent(
    llm_engine=llm_engine,
    tools=[],
    max_iterations=10,
)

function = """import numpy as np
def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w"""

task = """Just answer with "2"
"""


def interact_with_agent(prompt):
    full_prompt = task.replace("<<function>>", prompt)
    messages = []
    messages.append(ChatMessage(role="user", content=full_prompt))
    yield messages
    for msg in stream_to_gradio(agent, full_prompt):
        messages.append(msg)
        yield messages + [
            ChatMessage(role="assistant", content="⏳ Task not finished yet!")
        ]
    yield messages


with gr.Blocks(theme="soft") as demo:
    gr.Markdown(
        """### Python test generator
Write your function in the left textbox, and the agent on the right will generate tests for you!"""
    )
    with gr.Row():
        with gr.Column():
            text_input = gr.Textbox(
                lines=1, label="Your function to test", value=function
            )
            submit = gr.Button("Generate tests!")
        with gr.Column():
            chatbot = Chatbot(label="Agent", type="messages")

    submit.click(interact_with_agent, [text_input], [chatbot])

if __name__ == "__main__":
    demo.launch()

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /Users/aymeric/.cache/huggingface/token
Login successful
Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


======== New task ========
Just answer with "2"

Calling tool: 'final_answer' with arguments: {'answer': '2'}


In [2]:
!pip install gradio --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 10.9 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 12.6 MB/s eta 0:00:00
  Attempting uninstall: gradio-client
    Found existing installation: gradio_client 1.0.2
    Uninstalling gradio_client-1.0.2:
      Successfully uninstalled gradio_client-1.0.2
  Attempting uninstall: gradio
    Found existing installation: gradio 4.37.2
    Uninstalling gradio-4.37.2:
      Successfully uninstalled gradio-4.37.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lavague-gradio 0.2.4 requires gradio==4.26.0, but you have gradio 4.38.1 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [23]:
from huggingface_hub import InferenceClient

client = InferenceClient("meta-llama/Meta-Llama-3.1-70B-Instruct")

messages = [
    {
        "role": "system",
        "content": """
Environment: ipython
Tools: brave_search, final_answer

Cutting Knowledge Date: 01 March 2023
Today's Date: 13 July 2024

You are a helpful Assistant.
Please help me answer this question by laying out a code snippet.
Call the tool final_answer with the answer, as in final_answer(your_answer), to give your answer

Now let's start.
""",
    },
    {"role": "user", "content": "What is 2+2?"},
]

output = client.chat_completion(messages=messages)
print(output.choices[0].message.content)

def calculate_sum():
    a = 2
    b = 2
    return a + b

final_answer(calculate_sum())


In [2]:
import gradio as gr
from transformers import (
    load_tool,
    ReactCodeAgent,
    HfEngine,
)

# Import tool from Hub

llm_engine = HfEngine("meta-llama/Meta-Llama-3.1-70B-Instruct")

# Initialize the agent with the image generation tool
agent = ReactCodeAgent(tools=[], llm_engine=llm_engine)

agent.run("What is the 6th FIbonacci number?")

======== New task ========
What is the 6th FIbonacci number?
Error in generating llm output: 424 Client Error: Failed Dependency for url: https://api-inference.huggingface.co/models/meta-llama/Meta-Llama-3.1-70B-Instruct/v1/chat/completions (Request ID: KdRTRtrMtd1FZWFjoL7Og)

Request failed during generation: Server error: '^'.
Traceback (most recent call last):
  File "/Users/aymeric/venvs/disposable/lib/python3.10/site-packages/huggingface_hub/utils/_errors.py", line 304, in hf_raise_for_status
    response.raise_for_status()
  File "/Users/aymeric/venvs/disposable/lib/python3.10/site-packages/requests/models.py", line 1021, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 424 Client Error: Failed Dependency for url: https://api-inference.huggingface.co/models/meta-llama/Meta-Llama-3.1-70B-Instruct/v1/chat/completions

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "

'Error in generating final llm output: 422 Client Error: Unprocessable Entity for url: https://api-inference.huggingface.co/models/meta-llama/Meta-Llama-3.1-70B-Instruct/v1/chat/completions (Request ID: ZfElb-wYqbFwwm6k7BaYP).'

In [2]:
import gradio as gr
from transformers import (
    load_tool,
    ReactCodeAgent,
    HfEngine,
    stream_to_gradio,
)

# Import tool from Hub
image_generation_tool = load_tool("m-ric/text-to-image")

llm_engine = HfEngine("meta-llama/Meta-Llama-3-70B-Instruct")

# Initialize the agent with the image generation tool
agent = ReactCodeAgent(tools=[image_generation_tool], llm_engine=llm_engine)


def interact_with_agent(task):
    messages = []
    messages.append(gr.ChatMessage(role="user", content=task))
    yield messages
    for msg in stream_to_gradio(agent, task):
        messages.append(msg)
        yield messages + [
            gr.ChatMessage(role="assistant", content="⏳ Task not finished yet!")
        ]
    yield messages


with gr.Blocks() as demo:
    text_input = gr.Textbox(
        lines=1,
        label="Chat Message",
        value="Make me a picture of the Statue of Liberty.",
    )
    submit = gr.Button("Run illustrator agent!")
    chatbot = gr.Chatbot(
        label="Agent",
        type="messages",
        avatar_images=(
            None,
            "https://em-content.zobj.net/source/twitter/53/robot-face_1f916.png",
        ),
    )
    submit.click(interact_with_agent, [text_input], [chatbot])

if __name__ == "__main__":
    demo.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


======== New task ========
Make me a picture of the Statue of Liberty.
==== Agent is executing the code below:
image = image_generator(prompt="A high-resolution, photorealistic image of the Statue of Liberty at sunset with a bright blue sky and a slight mist in the background.")
final_answer(image)
====
Print outputs:

>>> Final answer:
/var/folders/6m/9b1tts6d5w960j80wbw9tx3m0000gn/T/tmps6vlpwnv/a8350500-9033-4138-a09a-d7f49d65457b.png


message ChatMessage(role='assistant', content='Thought: I will use the `image_generator` tool to create an image of the Statue of Liberty. I will provide a descriptive prompt to ensure a high-quality image.\n\n', metadata=Metadata(title=None))
message ChatMessage(role='assistant', content='```py\nimage = image_generator(prompt="A high-resolution, photorealistic image of the Statue of Liberty at sunset with a bright blue sky and a slight mist in the background.")\nfinal_answer(image)\n```', metadata={'title': '🛠️ Used tool code interpreter'})
